In [1]:
#-*-coding:utf-8-*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/gulim.ttc"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

In [45]:
# 경로 설정
path = 'csv/'

train = pd.read_csv(path + 'train.csv', encoding='utf-8')
test = pd.read_csv(path + 'test.csv',  encoding='utf-8')
submission = pd.read_csv(path + 'sample_submission.csv',  encoding='utf-8')

In [46]:
# 데이터 오류
train[train['일자']=='2018-06-01']['요일']='금'

In [47]:
# 요일 범주화
train['요일'] = train['요일'].map({'월':0.01, '화':0.02, '수':0.03, '목':0.04, '금':0.05})
test['요일'] = test['요일'].map({'월':0.01, '화':0.02, '수':0.03, '목':0.04, '금':0.05})

In [48]:
# 일자 분리
train[['년','월','일']] = pd.DataFrame(train['일자'].str.split('-').tolist())
train = train.astype({'년':'int','월':'int','일':'int'})

test[['년','월','일']] = pd.DataFrame(test['일자'].str.split('-').tolist())
test = test.astype({'년':'int','월':'int','일':'int'})

In [ ]:
train.groupby(by='년').

In [49]:
# 실근무자수
train['실근무자수'] = train['본사정원수']- train['본사출장자수']- train['현본사소속재택근무자수']-train['본사휴가자수']
test['실근무자수'] = test['본사정원수']- test['본사출장자수']- test['현본사소속재택근무자수']-test['본사휴가자수']

# 중식 train 전처리

In [50]:
# 중식
train_ln = train[["월","요일","본사정원수","본사휴가자수","본사출장자수","본사시간외근무명령서승인건수","실근무자수","중식계"]]
test_ln = test[["월","요일","본사정원수","본사휴가자수","본사출장자수","본사시간외근무명령서승인건수","실근무자수"]]

In [51]:
# 요일 가중치
train_ln['요일가중치']=train['요일']
test_ln['요일가중치']=test['요일']

day_ln=round(train.groupby(by='요일').mean()['중식계']/100,1)

train_ln['요일가중치'] = train_ln['요일가중치'].map(day_ln)
test_ln['요일가중치'] = test_ln['요일가중치'].map(day_ln)

In [52]:
# 월별 가중치
train_ln['월별가중치']=train['월']
test_ln['월별가중치']=test['월']

week_ln=round(train.groupby(by='월').mean()['중식계']/100,1)

train_ln['월별가중치'] = train_ln['월'].map(week_ln)
test_ln['월별가중치'] = test_ln['월'].map(week_ln)

In [53]:
#상관관계
train_ln.corr()['중식계'] # 월 요일 실근무자수

월                -0.154664
요일               -0.731563
본사정원수            -0.115529
본사휴가자수           -0.391975
본사출장자수           -0.512680
본사시간외근무명령서승인건수    0.535611
실근무자수             0.286810
중식계               1.000000
요일가중치             0.759177
월별가중치             0.224519
Name: 중식계, dtype: float64

# 중식 모델

In [54]:
from pycaret.regression import *
py_ln = setup(data = train_ln, target = '중식계',train_size=0.8)

,Description,Value
0,session_id,5214
1,Target,중식계
2,Original Data,"(1205, 10)"
3,Missing Values,False
4,Numeric Features,8
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(964, 20)"


In [55]:
best3_ln = compare_models(sort='MAE',n_select=3)
model_ln = blend_models(estimator_list = best3_ln) # 앙상블 모델
tune_model_ln=tune_model(model_ln)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,70.38,8909,94.39,0.8034,-0,0.0836
1,72.8,9656,98.26,0.7863,-0,0.0923
2,64.38,7841,88.55,0.8477,-0,0.0722
3,72.52,8764,93.62,0.8021,-0,0.0885
4,59.25,7213,84.93,0.838,-0,0.0734
5,70.82,8832,93.98,0.8086,-0,0.086
6,76.33,1.101e+04,104.9,0.7553,-0,0.0955
7,68.51,8028,89.6,0.8189,-0,0.0803
8,57.66,6226,78.91,0.8703,-0,0.0718
9,68.27,8054,89.74,0.7908,-0,0.0913


In [56]:
print(best3_ln)
print(tune_model_ln)

[GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=5214, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False), ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fra

In [57]:
model_fin_ln = finalize_model(tune_model_ln)
pred_ln = predict_model(model_fin_ln,test_ln)
pred_ln

,월,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,실근무자수,요일가중치,월별가중치,Label
0,1,0.03,2983,88,182,5,2355.0,9.1,9.3,995.969771
1,1,0.04,2983,104,212,409,2319.0,8.2,9.3,940.834918
2,1,0.05,2983,270,249,0,2170.0,6.5,9.3,601.963219
3,2,0.01,2924,108,154,538,2340.0,11.4,9.9,1205.419183
4,2,0.02,2924,62,186,455,2362.0,9.3,9.9,1016.044538
5,2,0.03,2924,59,199,5,2380.0,9.1,9.9,1000.901075
6,2,0.04,2924,61,211,476,2364.0,8.2,9.9,933.802197
7,2,0.05,2924,169,252,0,2247.0,6.5,9.9,676.487466
8,2,0.01,2924,88,174,690,2333.0,11.4,9.9,1263.268727
9,2,0.02,2924,94,183,542,2318.0,9.3,9.9,1031.943799


In [58]:
submission['중식계'] = pred_ln['Label']

# 석식 train 전처리

In [60]:
train_dn = train[["월","요일","본사정원수","본사휴가자수","본사출장자수","본사시간외근무명령서승인건수","실근무자수","석식계"]]
test_dn = test[["월","요일","본사정원수","본사휴가자수","본사출장자수","본사시간외근무명령서승인건수","실근무자수"]]

In [61]:
# 요일가중치
train_dn['요일가중치']=train_dn['요일']
test_dn['요일가중치']=test_dn['요일']

train_dn['요일가중치'] = train_dn['요일가중치'].map({0.01:5.3, 0.02:5.2, 0.03:3.6, 0.04:4.8, 0.05:4})
test_dn['요일가중치'] = test_dn['요일가중치'].map({0.01:5.3, 0.02:5.2, 0.03:3.6, 0.04:4.8, 0.05:4})

In [67]:
# 월별 가중치

train_dn['월별가중치']=train['월']
test_dn['월별가중치']=test['월']

week_dn=round(train.groupby(by='월').mean()['석식계']/100,1)

train_dn['월별가중치'] = train_dn['월'].map(week_dn)
test_dn['월별가중치'] = test_dn['월'].map(week_dn)

In [68]:
# 석식계
train_dn.corr()['석식계']

월                -0.127142
요일               -0.312112
본사정원수            -0.173852
본사휴가자수           -0.316894
본사출장자수           -0.188164
본사시간외근무명령서승인건수    0.571168
실근무자수             0.172373
석식계               1.000000
요일가중치             0.481371
월별가중치             0.253442
Name: 석식계, dtype: float64

# 석식 예측

In [71]:
from pycaret.regression import *
py_dn = setup(data = train_dn, target = '석식계',train_size=0.8)

,Description,Value
0,session_id,4149
1,Target,석식계
2,Original Data,"(1205, 10)"
3,Missing Values,False
4,Numeric Features,8
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(964, 20)"


In [72]:
best3_dn = compare_models(sort='MAE',n_select=3)
print(best3_dn)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,65.45,1.073e+04,102.5,0.4525,0,0.1266,0.501
rf,Random Forest Regressor,65.59,1.043e+04,101.1,0.468,0,0.1255,0.551
gbr,Gradient Boosting Regressor,65.7,9888,98.47,0.4952,0,0.1267,0.152
lightgbm,Light Gradient Boosting Machine,66.73,1.053e+04,101.4,0.4663,0,0.1275,0.293
lasso,Lasso Regression,68.57,1.071e+04,102.2,0.4582,0,0.1315,0.022
ridge,Ridge Regression,68.6,1.078e+04,102.4,0.4555,0,0.1311,0.021
lr,Linear Regression,69.45,1.052e+04,101.2,0.4675,0,0.1343,1.346
lar,Least Angle Regression,69.46,1.052e+04,101.2,0.4674,0,0.1343,0.022
huber,Huber Regressor,70.47,1.187e+04,107.4,0.404,0,0.1398,0.07
knn,K Neighbors Regressor,71.16,1.161e+04,106.7,0.4059,0,0.1392,0.061


[ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=-1, oob_score=False,
                    random_state=4149, verbose=0, warm_start=False), RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=4149, verbose=0, warm_start=False), GradientBoostingRegr

In [73]:
model_dn = blend_models(estimator_list = best3_dn) # 앙상블 모델
tune_model_dn=tune_model(model_dn)
print(tune_model_dn)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,55.92,6465,80.4,0.5236,-0,0.109
1,65.32,9582,97.89,0.5195,-0,0.1269
2,65.73,9606,98.01,0.4214,-0,0.1196
3,67.92,9920,99.6,0.4757,-0,0.1179
4,58.43,7466,86.4,0.596,-0,0.1291
5,74.31,1.585e+04,125.9,0.3961,-0,0.1177
6,56.98,6187,78.66,0.5261,-0,0.141
7,73.25,1.385e+04,117.7,0.529,-0,0.1371
8,56.42,8233,90.74,0.5301,-0,0.1142
9,63.35,1.015e+04,100.8,0.5268,-0,0.1069


VotingRegressor(estimators=[('et',
                             ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0,
                                                 criterion='mse',
                                                 max_depth=None,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 max_samples=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100, n_jobs=-1,
                                                 oob_score=False,
                                  

In [74]:
model_fin_dn = finalize_model(tune_model_dn)
pred_dn = predict_model(model_fin_dn,test_dn)
pred_dn

,월,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,실근무자수,요일가중치,월별가중치,Label
0,1,0.03,2983,88,182,5,2355.0,3.6,4.2,360.201065
1,1,0.04,2983,104,212,409,2319.0,4.8,4.2,430.273703
2,1,0.05,2983,270,249,0,2170.0,4.0,4.2,267.998843
3,2,0.01,2924,108,154,538,2340.0,5.3,5.1,577.069011
4,2,0.02,2924,62,186,455,2362.0,5.2,5.1,508.319151
5,2,0.03,2924,59,199,5,2380.0,3.6,5.1,398.940179
6,2,0.04,2924,61,211,476,2364.0,4.8,5.1,507.152450
7,2,0.05,2924,169,252,0,2247.0,4.0,5.1,343.263349
8,2,0.01,2924,88,174,690,2333.0,5.3,5.1,646.392695
9,2,0.02,2924,94,183,542,2318.0,5.2,5.1,562.433179


In [75]:
submission['석식계'] = pred_dn['Label']

In [76]:
submission

,일자,중식계,석식계
0,2021-01-27,995.969771,360.201065
1,2021-01-28,940.834918,430.273703
2,2021-01-29,601.963219,267.998843
3,2021-02-01,1205.419183,577.069011
4,2021-02-02,1016.044538,508.319151
5,2021-02-03,1000.901075,398.940179
6,2021-02-04,933.802197,507.152450
7,2021-02-05,676.487466,343.263349
8,2021-02-08,1263.268727,646.392695
9,2021-02-09,1031.943799,562.433179


In [77]:
submission.to_csv('result/result_0714_2.csv', index=False)